In [ ]:
# "kayhal/naruda_poems"

In [ ]:
!pip install datasets

In [ ]:
dataset

Dataset({
    features: ['Title', 'Poem'],
    num_rows: 11
})

In [ ]:
from datasets import load_dataset
dataset = load_dataset('checkai/instruction-poems', split='train[:500]')
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google-t5/t5-small")


def preprocess_function(dataset):
  inputs = ["give a poem related to: " + title for title in dataset["INSTRUCTION"]]
  model_inputs = tokenizer(inputs, max_length=512, truncation=True)

  with tokenizer.as_target_tokenizer():
      labels = tokenizer(dataset["RESPONSE"], max_length=1024, truncation=True)

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs


dataset = dataset.map(preprocess_function, batched=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3866: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
dataset

Dataset({
    features: ['INSTRUCTION', 'RESPONSE', 'SOURCE', 'METADATA', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 13854
})

In [ ]:
# prompt: dataset remove columns

dataset = dataset.remove_columns(['INSTRUCTION', 'RESPONSE', 'SOURCE', 'METADATA'])

In [ ]:
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainer

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:

from transformers import Seq2SeqTrainingArguments


In [ ]:
# !pip install accelerate -U

In [ ]:
training_args = Seq2SeqTrainingArguments(
  output_dir="./results",
  evaluation_strategy="epoch",
  learning_rate=2e-5,
  per_device_train_batch_size=2,
  per_device_eval_batch_size=2,
  weight_decay=0.01,
  save_total_limit=3,
  num_train_epochs=2,
  logging_steps = 2,
)


In [ ]:

trainer = Seq2SeqTrainer(
  model=model,
  args=training_args,
  train_dataset=dataset,
  eval_dataset=dataset,
  data_collator=data_collator,
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,4.603000,4.635003
2,4.889300,4.589044


TrainOutput(global_step=500, training_loss=5.030121725082397, metrics={'train_runtime': 95.3188, 'train_samples_per_second': 10.491, 'train_steps_per_second': 5.246, 'total_flos': 15909534498816.0, 'train_loss': 5.030121725082397, 'epoch': 2.0})

In [ ]:
model.to('cuda')

In [ ]:
input_ids = tokenizer("give a poem related to: drunk morning with pines", return_tensors="pt").input_ids
outputs = model.generate(input_ids.to('cuda'), max_new_tokens=128)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))